In [13]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

with open("casesvsadmissions.json", "rt") as INFILE:
    jsondata=json.load(INFILE)
    
def wrangle_data(data):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=data['data'] # data['data'] is a list
    # datalist

    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    # dates

    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    # print (startdate, ' to ', enddate)


    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cum_Cases_By_Publish_Date', 'cum_Admissions'])
    # timeseriesdf

    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cum_Cases_By_Publish_Date', 'cum_Admissions']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                timeseriesdf.loc[date, column]=value

    # fill in any remaining "holes" due to missing dates
    timeseriesdf.fillna(0.0, inplace=True)
    return timeseriesdf


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other # forces the redraw
    whichwalk.value=current # now we can change it back
    
    # our API access function. This will be called by the button when it is clicked
def access_api(Button):
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    structure = {"date": "date",
                 "cum_Cases_By_Publish_Date": "cumCasesByPublishDate", 
                  "cum_Admissions": "cumAdmissions"}

    mydash = Cov19API(filters=filters, structure=structure)
    dailyadmissions = mydash.get_json()
    # print(dailyadmissions)
    apidata=access_api()
    apibutton.disabled=True
    global df
    refresh_graph()
    apibutton.icon="check"
    


# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)
#return {} # return data read from the API

agecols=wdg.SelectMultiple(
    options=['cum_Cases_By_Publish_Date', 'cum_Admissions'], # options available
    value=['cum_Cases_By_Publish_Date', 'cum_Admissions'], # initial value
    rows=2, # rows of the selection box
    description='Cumulative Cases VS Cumulative Hospital Admissions',
    disabled=False
)

def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(y=list(graphcolumns),logy= True) # graphcolumns is a tuple - we need a list
        #plt.xscale('log')
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, output)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

SelectMultiple(description='Cumulative Cases VS Cumulative Hospital Admissions', index=(0, 1), options=('cum_C…

Output()